In [ ]:
import re
import numpy as np
import json
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
minotaur=[[0.0, 0.2, 0.2, 0.7, 0.8, 2.4, 6.0, 3.1, 6.2, 41.2, 62.6], [0.1, 0.4, 0.7, 2.3, 1.0, 5.3, 2.8, 15.9, 52.8, 64.1, 93.2], [0.3, 0.9, 0.6, 0.9, 3.2, 4.8, 5.3, 24.5, 79.4, 76.8, 92.0], [0.4, 0.6, 1.0, 2.7, 3.1, 5.2, 41.4, 59.5, 80.0, 92.1, 96.0], [1.1, 1.8, 1.3, 10.2, 7.9, 34.9, 74.9, 87.4, 93.3, 106.5, 113.0], [1.0, 1.9, 2.3, 9.9, 33.4, 52.3, 85.0, 90.9, 106.1, 109.6, 116.5], [0.5, 9.1, 7.8, 23.5, 42.6, 88.2, 91.1, 99.7, 109.4, 122.6, 120.7], [5.1, 13.3, 29.2, 51.0, 84.0, 95.3, 102.4, 111.7, 117.2, 126.3, 132.7], [24.1, 37.5, 38.3, 83.8, 91.7, 101.2, 114.8, 111.2, 128.1, 130.7, 144.3], [36.2, 56.1, 67.1, 93.6, 97.9, 112.4, 113.4, 126.8, 138.9, 135.9, 150.8], [33.6, 77.4, 84.9, 97.2, 105.7, 113.9, 126.4, 139.0, 147.4, 144.9, 157.3]]
pure_pos=[0.0, 0.0, 0.5, 0.6, 8.5, 50.3, 97.1, 110.7, 121.8, 139.1, 159.8]
minotaur = np.array(minotaur)
pure_pos = np.array(pure_pos)

In [ ]:
minotaur_max = np.ceil(np.max(minotaur))
norm = matplotlib.colors.Normalize(vmin=0, vmax=minotaur_max)

grid = dict(height_ratios=[minotaur.shape[0],1], width_ratios=[1,minotaur.shape[0], 0.5 ])
fig, axes = plt.subplots(ncols=3, nrows=2, gridspec_kw = grid, figsize=[6.4*1.2, 5.9*1.2])
axes[0,1].imshow(minotaur, cmap='Reds', aspect='auto', interpolation='nearest', origin='lower', norm=norm)
axes[0,0].imshow(pure_pos.reshape((11,1)), cmap='Reds', aspect='auto', interpolation='nearest', origin='lower', norm=norm)
axes[1,1].imshow(pure_pos.reshape((1,11)), cmap='Reds', aspect='auto', interpolation='nearest', origin='lower', norm=norm)
axes[1,0].axis("off")
axes[1,2].axis("off")
for ax in [axes[0,1], axes[0,0], axes[1,1]]:
    ax.set_xticks([]); ax.set_yticks([])
fig.colorbar(matplotlib.cm.ScalarMappable(cmap="Reds", norm=norm), cax=axes[0,2])

axes[1,1].set_xlabel(r"$\beta_w$")
axes[0,0].set_ylabel(r"$\beta_s$")
axes[0,1].set_xticks(np.linspace(0,10,6), np.round(np.linspace(0,1,6),2))
axes[0,1].set_yticks(np.linspace(0,10,6), np.round(np.linspace(0,1,6),2))
axes[0,1].set_title("Minotaur")
axes[0,0].set_title("Ouroboros\n Praos")
axes[1,2].set_title("Bitcoin", x=0.5, y=0.1)

plt.savefig("tmp10.pdf", bbox_inches='tight')